Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [2]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [3]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [16]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

In [17]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   13.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [18]:
grid_search.best_score_

0.8798133022170361

In [7]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [3]:
import pandas as pd
train = pd.read_csv('./module3-document-classification/data/train.csv')
test = pd.read_csv('./module3-document-classification/data/test.csv')

train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [8]:
train['category'].value_counts()

1    1637
2     449
3     300
4     200
Name: category, dtype: int64

In [4]:
import spacy

nlp = spacy.load("en_core_web_lg")
tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)

In [5]:
def tokenize(text):
    tokenized = tokenizer(text.lower())
    stripped_tokens = []
    for token in tokenized:
        if token.is_stop == False and token.is_punct == False:
            append = token.lemma_
            stripped_tokens.append(append)
    return (stripped_tokens)

tokenize(train['description'][0])

['marriage',
 '13',
 '18',
 'year',
 'old',
 'bourbons.',
 'mature',
 'elegant',
 'whiskey,',
 'silky',
 'texture',
 'easy',
 'embrace',
 'splash',
 'water.',
 'balance',
 'note',
 'honeyed',
 'vanilla,',
 'soft',
 'caramel,',
 'basket',
 'complex',
 'orchard',
 'fruit,',
 'blackberry,',
 'papaya,',
 'dust',
 'cocoa',
 'nutmeg;',
 'smooth',
 'finish.',
 'sophisticated,',
 'stylish,',
 'well-defined',
 'flavors.',
 'classic!']

### Define Pipeline Components

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

tfidfVectorizer = TfidfVectorizer(
    #tokenizer=tokenize,
    ngram_range=(1,2),
)
randomForestClassifier = RandomForestClassifier(
    n_jobs=-2,
    oob_score=True,
)

parameters = {
    'RandomForestClassifier__n_estimators': (7,70,700,),
    'RandomForestClassifier__min_samples_leaf': (37,137,),
    'TfidfVectorizer__stop_words': ('english', None),
    'TfidfVectorizer__min_df': (.02, .15, 1, 5, 0,),
    'TfidfVectorizer__max_df': (.9, .95, .99, 1.0,),
    'TfidfVectorizer__max_features': (500,1000,),
}

pipeline = Pipeline([
    ('TfidfVectorizer', tfidfVectorizer),
    ('RandomForestClassifier', randomForestClassifier)
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model.

In [100]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

grid_search = RandomizedSearchCV(
    pipeline,
    parameters,
    n_iter=64,
    cv=4, 
    n_jobs=1,
    verbose=3,
)
grid_search.fit(train['description'], train['category'])

idfVectorizer__min_df=0.15, TfidfVectorizer__max_features=500, TfidfVectorizer__max_df=0.95, RandomForestClassifier__n_estimators=70, RandomForestClassifier__min_samples_leaf=37, score=0.703, total=   2.7s
[CV] TfidfVectorizer__stop_words=None, TfidfVectorizer__min_df=0.15, TfidfVectorizer__max_features=500, TfidfVectorizer__max_df=0.95, RandomForestClassifier__n_estimators=70, RandomForestClassifier__min_samples_leaf=37 
[CV]  TfidfVectorizer__stop_words=None, TfidfVectorizer__min_df=0.15, TfidfVectorizer__max_features=500, TfidfVectorizer__max_df=0.95, RandomForestClassifier__n_estimators=70, RandomForestClassifier__min_samples_leaf=37, score=0.664, total=   2.8s
[CV] TfidfVectorizer__stop_words=None, TfidfVectorizer__min_df=0.15, TfidfVectorizer__max_features=500, TfidfVectorizer__max_df=0.95, RandomForestClassifier__n_estimators=700, RandomForestClassifier__min_samples_leaf=37 
[CV]  TfidfVectorizer__stop_words=None, TfidfVectorizer__min_df=0.15, TfidfVectorizer__max_features=500, 

RandomizedSearchCV(cv=4, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('TfidfVectorizer',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                             

In [101]:
grid_search.best_params_

{'TfidfVectorizer__stop_words': 'english',
 'TfidfVectorizer__min_df': 0,
 'TfidfVectorizer__max_features': 500,
 'TfidfVectorizer__max_df': 0.99,
 'RandomForestClassifier__n_estimators': 7,
 'RandomForestClassifier__min_samples_leaf': 37}

In [25]:
parameters = {
    'RandomForestClassifier__n_estimators': (270,),
    'RandomForestClassifier__min_samples_leaf': (17,7,4),
    'TfidfVectorizer__min_df': (.02,),
    'TfidfVectorizer__max_features': (500,),
}

pipeline = Pipeline([
    ('TfidfVectorizer', tfidfVectorizer),
    ('RandomForestClassifier', randomForestClassifier)
])

In [26]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipeline,
    parameters,
    cv=3, 
    n_jobs=1,
    verbose=3
)
grid_search.fit(train['description'], train['category'])

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] RandomForestClassifier__min_samples_leaf=17, RandomForestClassifier__n_estimators=270, TfidfVectorizer__max_features=500, TfidfVectorizer__min_df=0.02 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  RandomForestClassifier__min_samples_leaf=17, RandomForestClassifier__n_estimators=270, TfidfVectorizer__max_features=500, TfidfVectorizer__min_df=0.02, score=0.743, total=   6.2s
[CV] RandomForestClassifier__min_samples_leaf=17, RandomForestClassifier__n_estimators=270, TfidfVectorizer__max_features=500, TfidfVectorizer__min_df=0.02 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s
[CV]  RandomForestClassifier__min_samples_leaf=17, RandomForestClassifier__n_estimators=270, TfidfVectorizer__max_features=500, TfidfVectorizer__min_df=0.02, score=0.745, total=   6.4s
[CV] RandomForestClassifier__min_samples_leaf=17, RandomForestClassifier__n_estimators=270, Tfid

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('TfidfVectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                               

In [32]:
grid_search.best_params_

{'RandomForestClassifier__min_samples_leaf': 4,
 'RandomForestClassifier__n_estimators': 270,
 'TfidfVectorizer__max_features': 500,
 'TfidfVectorizer__min_df': 0.02}

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy.

In [102]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [103]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [104]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [105]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
import datetime

dt = f'{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}'
submission.to_csv(f'./module3-document-classification/data/submission{dt}.csv', index=False)

## Challenge

You're trying to achienve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [9]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [10]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0]
}

In [12]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [13]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:   23.7s finished
/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [14]:
grid_search.best_score_

0.8786464410735122

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [15]:
from sklearn.decomposition import TruncatedSVD

truncatedSVD = TruncatedSVD(
    n_components=100, 
    algorithm='randomized',
    n_iter=10
)

parameters = {
    'RandomForestClassifier__n_estimators': (70,),
    'RandomForestClassifier__max_depth': (None,),
    'RandomForestClassifier__min_samples_leaf': (7,),
    'TfidfVectorizer__min_df': (.02,),
    'TfidfVectorizer__max_features': (500,),
}

pipeline = Pipeline([
    ('TfidfVectorizer', tfidfVectorizer),
    ('TruncatedSVD', truncatedSVD),
    ('RandomForestClassifier', randomForestClassifier)
])


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model.

In [74]:
grid_search = GridSearchCV(
    pipeline,
    parameters,
    cv=3, 
    n_jobs=1,
    verbose=3
)
grid_search.fit(train['description'], train['category'])

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] RandomForestClassifier__max_depth=None, RandomForestClassifier__min_samples_leaf=7, RandomForestClassifier__n_estimators=70, TfidfVectorizer__max_features=500, TfidfVectorizer__min_df=0.02 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  RandomForestClassifier__max_depth=None, RandomForestClassifier__min_samples_leaf=7, RandomForestClassifier__n_estimators=70, TfidfVectorizer__max_features=500, TfidfVectorizer__min_df=0.02, score=0.778, total=   6.6s
[CV] RandomForestClassifier__max_depth=None, RandomForestClassifier__min_samples_leaf=7, RandomForestClassifier__n_estimators=70, TfidfVectorizer__max_features=500, TfidfVectorizer__min_df=0.02 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s
[CV]  RandomForestClassifier__max_depth=None, RandomForestClassifier__min_samples_leaf=7, RandomForestClassifier__n_estimators=70, TfidfVectorizer__max_features=500, T

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('TfidfVectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                               

In [75]:
grid_search.best_params_

{'RandomForestClassifier__max_depth': None,
 'RandomForestClassifier__min_samples_leaf': 7,
 'RandomForestClassifier__n_estimators': 70,
 'TfidfVectorizer__max_features': 500,
 'TfidfVectorizer__min_df': 0.02}

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy.

In [66]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [67]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [68]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,3
2,1390,1
3,1024,1
4,1902,1


In [69]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

dt = f'{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}'
submission.to_csv(f'./module3-document-classification/data/submission{dt}.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets.

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [112]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [26]:
len(X) == len(data.data)

True

In [22]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [23]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9918319719953326

## Follow Along

In [7]:
def w2v(col):
    vecs = []
    for desc in col:
        doc = nlp(desc)
        vecs.append(doc.vector)
    return (vecs)
w2v([train['description'][0]])

[array([-2.85943802e-02,  2.06485838e-01, -8.71750936e-02, -1.19492866e-01,
         9.76047292e-02,  1.47639196e-02,  5.54700717e-02, -1.20707422e-01,
        -3.37744504e-03,  1.58941615e+00, -1.00231625e-01,  6.32911772e-02,
        -9.23616216e-02, -2.78862435e-02, -1.46666154e-01, -1.47408605e-01,
        -6.91815391e-02,  1.26379716e+00, -1.10995278e-01, -6.84440508e-02,
        -9.96155515e-02,  8.43309239e-03, -2.82734949e-02, -5.33238007e-03,
         3.34358338e-04, -3.41439396e-02,  1.08375914e-01,  5.94048873e-02,
         1.51642635e-01, -2.04002455e-01,  1.91272907e-02,  7.04437569e-02,
        -3.80388424e-02, -6.65084273e-02,  1.26438960e-01, -1.39678836e-01,
         3.42864543e-02, -7.94737712e-02, -7.69885108e-02, -5.36400191e-02,
        -7.51534244e-03,  6.82128444e-02,  1.19646020e-01, -1.59290418e-01,
         9.51901525e-02,  1.45845219e-01, -2.18310863e-01, -9.96276457e-03,
         5.23019843e-02,  6.42457791e-03, -1.01173982e-01,  6.37993030e-03,
        -8.6

In [8]:
wvs = w2v(train['description'])

In [31]:
def process(series, tfidfVectorizer=None, truncatedSVD=None):
    wvs = w2v(series)
    wvs_df = pd.DataFrame(wvs)
    if tfidfVectorizer is None:
        tfidfVectorizer = TfidfVectorizer(
            #tokenizer=tokenize,
            ngram_range=(1,2),
            max_df=.99,
        )
        tfidf = tfidfVectorizer.fit_transform(series)
    else:
        tfidf = tfidfVectorizer.transform(series)
    if truncatedSVD is None:
        truncatedSVD = TruncatedSVD(
            n_components=100, 
            algorithm='randomized',
            n_iter=10
        )
        lsa = truncatedSVD.fit_transform(tfidf)
    else:
        lsa = truncatedSVD.transform(tfidf)
    tfidf_df = pd.DataFrame(lsa)
    lens = series.str.len()
    tokCounts = []
    for desc in series:
        tokCounts.append(len(tokenize(desc)))
    processed_df = wvs_df.join(tfidf_df, rsuffix='tfidf', lsuffix='wvs')
    processed_df['description_lengths'] = descLens
    processed_df['token_counts'] = tokCounts
    return (processed_df, tfidfVectorizer, truncatedSVD)

In [9]:
wvs_df = pd.DataFrame(wvs)
wvs_df.shape

(2586, 300)

In [10]:
train.shape

(2586, 3)

In [11]:
tfidf = tfidfVectorizer.fit_transform(train['description'])
tfidf.shape

(2586, 86283)

In [16]:
lsa = truncatedSVD.fit_transform(tfidf)
lsa.shape

(2586, 100)

In [17]:
tfidf_df = pd.DataFrame(lsa)
print(tfidf_df.shape)
tfidf_df.head()

(2586, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.134954,-0.067995,0.101689,-0.015063,-0.060480,-0.043316,0.092545,0.027669,-0.006990,0.014124,...,-0.022061,0.007763,0.001391,0.011398,-0.007397,-0.095201,0.010039,0.043599,-0.010715,0.004230
1,0.141275,-0.041420,-0.049230,0.036506,0.055922,-0.020099,-0.017803,0.007705,0.013313,0.006410,...,0.015456,-0.021648,0.016803,0.000809,-0.017058,-0.018892,0.009498,0.001579,0.032912,0.029081
2,0.158796,-0.069344,0.020795,0.006917,-0.117853,-0.036481,0.030613,0.012048,0.016597,0.029578,...,-0.007023,-0.031354,-0.012037,-0.029434,-0.013992,-0.023199,-0.016405,0.027376,0.054350,0.043372
3,0.166051,-0.047417,0.054438,0.060801,-0.003452,0.046175,-0.031772,0.042729,0.034755,0.056576,...,0.005252,0.046110,-0.007303,0.020945,-0.021382,-0.004965,-0.017955,0.022743,-0.000930,0.013111
4,0.095916,-0.026497,0.040762,-0.095652,-0.022530,0.051103,0.109655,0.133410,0.006761,0.128158,...,0.006965,-0.013717,-0.018804,-0.007546,-0.005857,-0.022501,-0.004640,-0.024224,-0.045805,0.021574


In [18]:
descLens = train['description'].str.len()
descLens

0        361
1        503
2        824
3        495
4        415
5        454
6        429
7        557
8        384
9        451
10       293
11       459
12       409
13       443
14       449
15       295
16       382
17       532
18      1255
19       501
20       371
21       771
22       380
23       509
24       369
25       827
26       406
27       475
28       375
29       377
        ... 
2556     434
2557     462
2558     362
2559     281
2560     827
2561     408
2562     455
2563     491
2564     359
2565     527
2566     412
2567     421
2568     453
2569     440
2570     378
2571     702
2572     365
2573     324
2574     488
2575     683
2576     323
2577     492
2578     463
2579     402
2580     342
2581     252
2582     442
2583     431
2584     327
2585     424
Name: description, Length: 2586, dtype: int64

In [20]:
tokCounts = []
for desc in train['description']:
    tokCounts.append(len(tokenize(desc)))

In [21]:
processed_df = wvs_df.join(tfidf_df, rsuffix='tfidf', lsuffix='wvs')
processed_df['description_lengths'] = descLens
processed_df['token_counts'] = tokCounts
processed_df.shape

(2586, 402)

In [32]:
pdf, tfidfVectorizer, truncatedSVD = process(train['description'])

In [33]:
pdf_test = process(test['description'], tfidfVectorizer=tfidfVectorizer, truncatedSVD=truncatedSVD)

In [37]:
pdf_test, _1, _2 = pdf_test

In [34]:
pdf.shape

(2586, 402)

In [38]:
pdf_test.shape

(288, 402)

In [50]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(pdf, train['category'], train_size=0.9)


xgbc_model = XGBClassifier(  
    random_state=1,
    n_jobs=-2,
    n_estimators=2700,
    min_samples_leaf=7,
    #max_depth=7,
    learning_rate=.025,
    verbosity=1
)

eval_set = [(X_train, y_train),
            (X_val, y_val)]

xgbc_model.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds=50)


[0]	validation_0-merror:0.129781	validation_1-merror:0.158301
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.128062	validation_1-merror:0.162162
[2]	validation_0-merror:0.122905	validation_1-merror:0.158301
[3]	validation_0-merror:0.123335	validation_1-merror:0.150579
[4]	validation_0-merror:0.11517	validation_1-merror:0.135135
[5]	validation_0-merror:0.109583	validation_1-merror:0.146718
[6]	validation_0-merror:0.109153	validation_1-merror:0.142857
[7]	validation_0-merror:0.108294	validation_1-merror:0.142857
[8]	validation_0-merror:0.105716	validation_1-merror:0.142857
[9]	validation_0-merror:0.105286	validation_1-merror:0.127413
[10]	validation_0-merror:0.09841	validation_1-merror:0.127413
[11]	validation_0-merror:0.09798	validation_1-merror:0.131274
[12]	validation_0-merror:0.095832	validation_1-merror:0.131274
[13]	validation_0-merror:0.0898

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, min_samples_leaf=7, missing=None,
              n_estimators=270, n_jobs=-2, nthread=None,
              objective='multi:softprob', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [51]:
preds = xgbc_model.predict(pdf_test)
preds.shape

(288,)

In [52]:
import datetime

submission = pd.DataFrame({'id': test['id'], 'category':preds})
submission['category'] = submission['category'].astype('int64')
dt = f'{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}'
submission.to_csv(f'./module3-document-classification/data/submission{dt}.csv', index=False)

In [0]:

randomForestClassifier = RandomForestClassifier(
    n_jobs=-2,
)

parameters = {
    'RandomForestClassifier__n_estimators': (17,71,170,710,),
    'RandomForestClassifier__max_depth': (7,17,37,70,),
    'RandomForestClassifier__min_samples_leaf': (2,4,7,11,),
}

# parameters = {
#     'RandomForestClassifier__n_estimators': (7,),
#     'RandomForestClassifier__min_samples_leaf': (37,),
# }

pipeline = Pipeline([
    ('RandomForestClassifier', randomForestClassifier)
])

grid_search = GridSearchCV(
    pipeline,
    parameters,
    cv=4, 
    n_jobs=1,
    verbose=3
)
grid_search.fit(processed_df, train['category'])

In [0]:
grid_search.best_params_

In [0]:
# Predictions on test sample
pred = grid_search.predict(w2v(test['description']))
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')
dt = f'{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}'
submission.to_csv(f'./module3-document-classification/data/submission{dt}.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on